In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Loading Data

In [ ]:
BASE_DIR = Path("/kaggle/input/playground-series-s3e2/")

train = pd.read_csv(BASE_DIR / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_DIR / "test.csv").drop(columns="id")

# Preprocessing

In [ ]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df.head()

# Feature Engineering

## Diabetes features

In [ ]:
def diabetes_indicator(avg_glucose_level):
    if avg_glucose_level <= 99:
        return "normal"
    elif avg_glucose_level >= 100 and avg_glucose_level <= 125:
        return "prediabetic"
    elif avg_glucose_level <= 200:
        return "type1"
    else:
        return "type2"

df.avg_glucose_level.map(diabetes_indicator).value_counts()

In [ ]:
df["diabetes"] = df.avg_glucose_level.map(diabetes_indicator)
df.head(3)

## BMI features 

In [ ]:
def bmi_level(bmi):
    if bmi < 18.5:
        return "underweight"
    elif bmi >= 18.5 and bmi < 25.0:
        return "healthy"
    elif bmi >= 25.0 and bmi < 30.0:
        return "overweight"
    elif bmi >= 30.0 and bmi < 35.0:
        return "obese_class1"
    elif bmi >= 35.0 and bmi < 40.0:
        return "obese_class2"
    elif bmi >= 40.0:
        return "obese_class3"

In [ ]:
df["bmi_level"] = df.bmi.map(bmi_level)
df.head(3)

# One hot encoding

In [ ]:
df = pd.get_dummies(df)
df.head(3)

# Preprocessing v2

In [ ]:
X = df.iloc[:-len(test), :]
test_new = df.iloc[-len(test):, :]
y = train.stroke

In [ ]:
xgb_params = {'n_estimators': 462,
                 'max_depth': 4,
                 'learning_rate': 0.06316459163174232,
                 'min_child_weight': 1,
                 'gamma': 0.0005505025153597244,
                 'subsample': 0.63,
                 'colsample_bytree': 0.92,
                 'reg_alpha': 0.0002471761194169256,
                 'reg_lambda': 0.33806845755800485}

In [ ]:
xgb_model = xgb.XGBClassifier(**xgb_params)

In [ ]:
xgb_model.fit(X, y)

In [ ]:
y_pred = xgb_model.predict_proba(test_new)[:, 1]
y_pred

In [ ]:
# since we dropped id column from our test dataframe earlier but we need it for submission file so we're reloading it
test_idx = pd.read_csv(BASE_DIR / "test.csv").id
test_idx

In [ ]:
submission = pd.DataFrame(data={"id": test_idx, "stroke": y_pred})
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)